what: given raster bbox, and return all the tilenames overlapping 1degree by 1 degree global griging like copernicus dsm

In [1]:
from uvars import drc_dir,topoxcale_dir,dataops_dir

In [2]:
import os 
import sys 
from glob import glob
import rasterio


def get_profile(tif_path):
    with rasterio.open(tif_path) as src:
        return src.profile
    
sys.path.append(dataops_dir)
from dataops.rops import list2txt, gdalbuildvrt, gdalreproject,merge_tifs

In [3]:
files = glob(f"{drc_dir}/*.tif")
for fi in files:
    pr = get_profile(fi)
    epsg_code = str(pr['crs'])
    print(epsg_code)

EPSG:32733
EPSG:32733
EPSG:32733


In [4]:
txt = "/home/ljp238/Downloads/DRC_LiDAR_Roy2021/dtm.txt"
vrt = "/home/ljp238/Downloads/DRC_LiDAR_Roy2021/dtm.vrt"

In [5]:
list2txt(txt, files)
gdalbuildvrt(vrt, txt, epsgcode=4326)

In [6]:
output_tif = vrt = "/home/ljp238/Downloads/DRC_LiDAR_Roy2021/DRC_LiDAR_Roy2021_DTM.tif"
merge_tifs(files, output_tif)

In [7]:
fi = "/home/ljp238/Downloads/DRC_LiDAR_Roy2021/DRC_LiDAR_Roy2021_DTM.tif"
fo = "/home/ljp238/Downloads/DRC_LiDAR_Roy2021/DRC_LiDAR_Roy2021_DTM_4326.tif"
gdalreproject(fi, fo, res=1/3600, epsgcode=4326)

Output file /home/ljp238/Downloads/DRC_LiDAR_Roy2021/DRC_LiDAR_Roy2021_DTM_4326.tif already exists. Skipping reprojection.


In [10]:
import geopandas as gpd
from shapely.geometry import box
import rasterio

fo = "/home/ljp238/Downloads/DRC_LiDAR_Roy2021/DRC_LiDAR_Roy2021_DTM_4326.tif"
vfpath = "/home/ljp238/Downloads/FABDEM_v1-2_tiles.geojson"

In [11]:
import geopandas as gpd
from shapely.geometry import box

# Function to get the bounding box of a tif file
def get_bbox(tif_path):
    import rasterio
    with rasterio.open(tif_path) as src:
        return box(*src.bounds)  # returns the bounding box as a shapely geometry

def get_tilenames_from_global_grid(tif_path, grid_path):
    g = gpd.read_file(grid_path)
    
    # Get the bounding box for the tif file
    bbox = get_bbox(tif_path)
    
    # Create a new column 'intersects' to check intersection with the bounding box
    g['intersects'] = g['geometry'].intersects(bbox)
    
    # Filter the rows where the geometry intersects with the bounding box
    intersecting_tiles = g[g['intersects']]
    
    # Extract the unique tile names
    unique_tile_names = intersecting_tiles['tile_name'].unique()
    
    # Extract the bounding boxes of the intersecting tiles
    unique_tile_bboxs = [tile.bounds for tile in intersecting_tiles.geometry]
    
    return unique_tile_names, unique_tile_bboxs,intersecting_tiles


In [12]:
tile_names, tile_bboxs ,intersecting_tiles = get_tilenames_from_global_grid(tif_path=fo,grid_path=vfpath)

In [13]:
tile_names

array(['S002E017', 'S002E018', 'S003E017', 'S003E018'], dtype=object)

In [13]:
# create a folder for data synth-data augumentation

In [14]:
from dataops import OpenTopgraphy as OT
import numpy as np
from dotenv import load_dotenv

available datasets on globaldem API
{'SRTM GL3 90m': 'SRTMGL3', 'SRTM GL1 30m': 'SRTMGL1', 'SRTM GL1 Ellipsoidal 30m': 'SRTMGL1_E', 'ALOS World 3D 30m': 'AW3D30', 'ALOS World 3D Ellipsoidal, 30m': 'AW3D30_E', 'Global Bathymetry SRTM15+ V2.1 500m': 'SRTM15Plus', 'NASADEM Global DEM': 'NASADEM', 'Copernicus Global DSM 30m': 'COP30', 'Copernicus Global DSM 90m': 'COP90', 'DTM 30m': 'EU_DTM', 'DTM 1000m': 'GEDI_L3', 'Global Bathymetry 500m': 'GEBCOIceTopo', 'Global Bathymetry Sub Ice 500m': 'GEBCOSubIceTopo'}


In [19]:
load_dotenv()
# Get the value of the environment variable
api_key = os.getenv("OPEN_TOPOGRAPHY_API_KEY")

In [20]:
dem_types = OT.dem_types

In [21]:
otdata_dir = "/home/ljp238/Downloads/DRC_LiDAR_Roy2021/OTdata/"
#varnames = np.array(list(dem_types.values()))

In [22]:
varnames = ['COP30']
for i in range(len(tile_names)):
    tile = tile_names[i]
    bbox = tile_bboxs[i]
    tile_dpath = os.path.join(otdata_dir,tile)
    os.makedirs(tile_dpath, exist_ok=True)
    for varname in varnames:
        tile_var_dpath = os.path.join(tile_dpath, varname)
        os.makedirs(tile_var_dpath, exist_ok=True)
        # OT.download_globaldem(outdir=tile_var_dpath,
        #                       bbox=bbox ,rpath=None, 
        #                       varname=varname, 
        #                       api_key=api_key)
    

In [23]:
import geopandas as gpd
from shapely.geometry import box

# Function to get the bounding box of a tif file
def get_bbox(tif_path):
    import rasterio
    with rasterio.open(tif_path) as src:
        return box(*src.bounds)  # returns the bounding box as a shapely geometry

def get_tilenames_from_global_grid(tif_path, grid_path):
    g = gpd.read_file(grid_path)
    
    # Get the bounding box for the tif file
    bbox = get_bbox(tif_path)
    
    # Create a new column 'intersects' to check intersection with the bounding box
    g['intersects'] = g['geometry'].intersects(bbox)
    
    # Filter the rows where the geometry intersects with the bounding box
    intersecting_tiles = g[g['intersects']]
    
    # Extract the unique tile names
    unique_tile_names = intersecting_tiles['tile_name'].unique()
    
    # Extract the bounding boxes of the intersecting tiles
    unique_tile_bboxs = [tile.bounds for tile in intersecting_tiles.geometry]
    
    return unique_tile_names, unique_tile_bboxs,intersecting_tiles


In [24]:
tile_names, tile_bboxs, gdf_tiles = get_tilenames_from_global_grid(tif_path=fo,grid_path=vfpath)

In [25]:
gdf_tiles

,tile_name,file_name,zipfile_name,geometry,intersects
15093,S002E017,S002E017_FABDEM_V1-2.tif,S10E010-N00E020_FABDEM_V1-2.zip,"POLYGON ((17 -2, 17 -1, 18 -1, 18 -2, 17 -2))",True
15094,S002E018,S002E018_FABDEM_V1-2.tif,S10E010-N00E020_FABDEM_V1-2.zip,"POLYGON ((18 -2, 18 -1, 19 -1, 19 -2, 18 -2))",True
15223,S003E017,S003E017_FABDEM_V1-2.tif,S10E010-N00E020_FABDEM_V1-2.zip,"POLYGON ((17 -3, 17 -2, 18 -2, 18 -3, 17 -3))",True
15224,S003E018,S003E018_FABDEM_V1-2.tif,S10E010-N00E020_FABDEM_V1-2.zip,"POLYGON ((18 -3, 18 -2, 19 -2, 19 -3, 18 -3))",True


In [26]:
gdf_tiles.head(1)

,tile_name,file_name,zipfile_name,geometry,intersects
15093,S002E017,S002E017_FABDEM_V1-2.tif,S10E010-N00E020_FABDEM_V1-2.zip,"POLYGON ((17 -2, 17 -1, 18 -1, 18 -2, 17 -2))",True


In [27]:
from dataops import GoogleEarthEngineDatasets as geed 

create patches for gee

In [28]:
from glob import glob
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
import numpy as np

In [29]:
GEEdata_dir = "/home/ljp238/Downloads/DRC_LiDAR_Roy2021/GEEdata" 
geepathes_dir = f"{GEEdata_dir}/patches"
geesentine1_dir = f"{GEEdata_dir}/S1"
os.makedirs(GEEdata_dir, exist_ok=True)
os.makedirs(geepathes_dir, exist_ok=True)
os.makedirs(geesentine1_dir, exist_ok=True)

In [33]:
gdf_tiles

,tile_name,file_name,zipfile_name,geometry,intersects
15093,S002E017,S002E017_FABDEM_V1-2.tif,S10E010-N00E020_FABDEM_V1-2.zip,"POLYGON ((17 -2, 17 -1, 18 -1, 18 -2, 17 -2))",True
15094,S002E018,S002E018_FABDEM_V1-2.tif,S10E010-N00E020_FABDEM_V1-2.zip,"POLYGON ((18 -2, 18 -1, 19 -1, 19 -2, 18 -2))",True
15223,S003E017,S003E017_FABDEM_V1-2.tif,S10E010-N00E020_FABDEM_V1-2.zip,"POLYGON ((17 -3, 17 -2, 18 -2, 18 -3, 17 -3))",True
15224,S003E018,S003E018_FABDEM_V1-2.tif,S10E010-N00E020_FABDEM_V1-2.zip,"POLYGON ((18 -3, 18 -2, 19 -2, 19 -3, 18 -3))",True


In [ ]:
patches_dir = "/home/ljp238/Downloads/DRC_LiDAR_Roy2021/GEEdata/patches"

In [41]:
for tilename in tile_names:
    subgdf = gdf_tiles[gdf_tiles['tile_name'] == tilename]
    subgdf = subgdf[['tile_name','geometry']]
    patches_tile_dir = os.path.join(patches_dir,tilename)
    os.makedirs(patches_tile_dir, exist_ok=True)
    tile_gpkg = os.path.join(patches_tile_dir,f"{tilename}.gpkg")
    if not os.path.isfile(tile_gpkg):
        subgdf.to_file(tile_gpkg, driver="GPKG")


In [38]:
type(subgdf)

geopandas.geodataframe.GeoDataFrame

In [43]:
vfiles =  glob(f"{patches_dir}/*/*.gpkg")
vpath =  vfiles[0]
vpath

'/home/ljp238/Downloads/DRC_LiDAR_Roy2021/GEEdata/patches/S002E017/S002E017.gpkg'

In [48]:
def vector2raster_properties(vpath,epsg = 4326, res=1/3600):

    vector = gpd.read_file(vpath)
    bbox = vector.total_bounds  # [minx, miny, maxx, maxy]
    if res is None:
        res = 1 / 3600  # Set resolution to 1/3600 degree
    #epsg = 4326  # EPSG:4326

    # Calculate the width and height of the raster
    width = int((bbox[2] - bbox[0]) / res) + 1
    height = int((bbox[3] - bbox[1]) / res) + 1
    # Define the transformation
    transform = from_origin(bbox[0], bbox[3], res, res)
    return width,height,transform

def make_dummy_raster(vpath, rpath,epsg = 4326,res=1/3600):
    width,height,transform = vector2raster_properties(vpath,epsg = epsg, res=res)
    # Create a new raster with uint8 data type and high compression
    with rasterio.open(rpath, 'w', driver='GTiff', height=height, width=width,
                       count=1, dtype='uint8', crs=f"EPSG:{epsg}", transform=transform,
                       compress='lzw') as dst:  # 'lzw' is a good compression algorithm for rasters
        # Generate random binary data (0s and 1s)
        random_data = np.random.choice([0, 1], size=(height, width), p=[0.5, 0.5])
        # Write the random data to the raster
        dst.write(random_data.astype('uint8'), 1)

dummy_tifs = []
for vapth in vfiles:
    print(vapth)
    tname = os.path.basename(vapth)[:-5]
    tname_dir = f"{geepathes_dir}/{tname}"
    os.makedirs(tname_dir, exist_ok=True)
    tname_fn = f"{tname_dir}/{tname}.tif"
    make_dummy_raster(vapth, tname_fn, epsg=4326, res=1/3600)
    dummy_tifs.append(tname_fn)

# there is something wrong here. all vfiles are different but all tname_fn are the same instead if different

/home/ljp238/Downloads/DRC_LiDAR_Roy2021/GEEdata/patches/S002E017/S002E017.gpkg
/home/ljp238/Downloads/DRC_LiDAR_Roy2021/GEEdata/patches/S002E018/S002E018.gpkg
/home/ljp238/Downloads/DRC_LiDAR_Roy2021/GEEdata/patches/S003E017/S003E017.gpkg
/home/ljp238/Downloads/DRC_LiDAR_Roy2021/GEEdata/patches/S003E018/S003E018.gpkg


In [77]:
from utils import GeoTile 

def gdaltindex(gpkg, tifs_dir):
    # do a rasterio as well
    cmd = f"gdaltindex -f GPKG {gpkg} {tifs_dir}/*.tif"
    os.system(cmd)

In [ ]:
def patchify_workflow(tifi,ps:int,st:int,cpus:int=20):
    gt = GeoTile(tifi)
    outdir = f"{os.path.dirname(tifi)}/{ps}_{st}"
    os.makedirs(outdir, exist_ok=True)

    gt.generate_tiles(
    output_folder=outdir,  # Folder to save tiles
    prefix="tile_",                # Optional: file name prefix
    suffix=None,                   # Optional: file name suffix
    save_tiles=True,               # Save tiles to disk
    save_transform=True,           # Save geotransform + CRS to txt
    out_bands=[1],                 # Read only the first band @@@
    image_format=".tif",           # Output format
    dtype="float32",               # Force output type
    tile_x=ps,                    # Tile width
    tile_y=ps,                    # Tile height
    stride_x=st,                  # Horizontal stride
    stride_y=st,                  # Vertical stride
    num_workers=cpus)                # Parallel threads (e.g., 8))
    tname = tifi.split('/')[-2]
    gpkg = f"{os.path.dirname(tifi)}/{tname}_tindex.gpkg"
    if not os.path.isfile(gpkg):
        gdaltindex(gpkg, tifs_dir=outdir)

In [ ]:
ps = 256
st = 256
tifi = dummy_tifs[0]

In [ ]:
for tifi in dummy_tifs:
    #patchify_workflow(tifi,ps,st,cpus=20)

In [74]:
gpkg

'/home/ljp238/Downloads/DRC_LiDAR_Roy2021/GEEdata/patches/S002E017/S002E017_tindex.gpkg'

In [70]:
os.remove(gpkg)